## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 📝 Override Result with Middleware

## Industry Use Case: Market Data Enrichment

This notebook demonstrates how middleware can **modify or override** agent responses.

| Feature | FSI Application |
|---------|-----------------|
| **Result Transformation** | Add compliance disclaimers |
| **Result Enrichment** | Append regulatory notices |
| **Result Validation** | Ensure responses meet standards |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")

In [ ]:
from collections.abc import AsyncIterable, Awaitable, Callable

from agent_framework import (
    AgentRunContext,
    AgentResponse,
    AgentResponseUpdate,
    ChatMessage,
    Role,
    TextContent,
    agent_middleware,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✅ All imports loaded")

## Define Market Data Tool

In [ ]:
def get_stock_price(symbol: str) -> str:
    """Get current stock price for a symbol."""
    prices = {"MSFT": 425.50, "AAPL": 178.25, "GOOGL": 142.80}
    price = prices.get(symbol.upper(), 100.00)
    return f"{symbol.upper()} is currently trading at ${price:.2f}"

print("✅ Tool defined: get_stock_price")

## Compliance Disclaimer Middleware

Appends regulatory disclaimers to all market data responses.

In [ ]:
@agent_middleware
async def compliance_disclaimer_middleware(
    context: AgentRunContext, 
    next: Callable[[AgentRunContext], Awaitable[None]]
) -> None:
    """Appends compliance disclaimers to all responses."""
    
    # Let the agent process normally first
    await next(context)
    
    # Add disclaimer to the result
    if context.result is not None:
        disclaimer = "\n\n⚠️ DISCLAIMER: This information is for educational purposes only and does not constitute financial advice."
        
        if context.is_streaming:
            # For streaming: create generator that appends disclaimer
            original_result = context.result
            
            async def add_disclaimer_stream() -> AsyncIterable[AgentResponseUpdate]:
                async for chunk in original_result:
                    yield chunk
                # Append disclaimer at end
                yield AgentResponseUpdate(contents=[TextContent(text=disclaimer)])
            
            context.result = add_disclaimer_stream()
        else:
            # For non-streaming: use .text property of response
            original_text = context.result.text or ""
            context.result = AgentResponse(
                messages=[ChatMessage(role=Role.ASSISTANT, text=original_text + disclaimer)]
            )

print("✅ Compliance middleware defined")

## Example 1: Non-streaming Result Override

In [ ]:
async def run_non_streaming_example():
    """Non-streaming response with disclaimer appended."""
    print("\n--- Example 1: Non-streaming Result Override ---\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="MarketDataAgent",
            instructions="You provide stock market data. Use the tool to get prices.",
            tools=[get_stock_price],
            middleware=[compliance_disclaimer_middleware],
        ) as agent,
    ):
        print("User: What's the price of MSFT?")
        result = await agent.run("What's the price of MSFT?")
        print(f"Agent: {result.text}")

await run_non_streaming_example()

## Example 2: Streaming Result Override

In [ ]:
async def run_streaming_example():
    """Streaming response with disclaimer appended at end."""
    print("\n--- Example 2: Streaming Result Override ---\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="MarketDataAgent",
            instructions="You provide stock market data. Use the tool to get prices.",
            tools=[get_stock_price],
            middleware=[compliance_disclaimer_middleware],
        ) as agent,
    ):
        print("User: What's the price of AAPL?")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream("What's the price of AAPL?"):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print()

await run_streaming_example()

## Key Takeaways

| Technique | When to Use | FSI Example |
|-----------|-------------|-------------|
| `await next(context)` first | Let agent process normally | Get original response |
| Modify `context.result` | Transform output | Add compliance disclaimers |
| `context.is_streaming` | Detect execution mode | Handle both patterns |
| Async generator | Stream overrides | Append text chunks |

## Next Steps
- **Shared State** (notebook 9) - Cross-middleware communication